# Parameters

The computational and physical parameters are handled by `class xcompact3d_toolbox.Parameters`. It is built on top of [Traitlets](https://traitlets.readthedocs.io/en/stable/index.html), which aims to make the parameters compatible with what xcompact3d expects, and also brings some advantages:

* Attributes are type-checked;
* Default values, restrictions and conections between related parameters are applied where necessary;
* 'On change' callbacks for validation and observation;
* Two-way linking with [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/).

In [ ]:
import numpy as np
import xcompact3d_toolbox as x3d

The first step is to establish numerical precision. Use `np.float64` if Xcompact3d was compiled with the flag `-DDOUBLE_PREC` (check the Makefile), use `np.float32` otherwise:

In [ ]:
x3d.param["mytype"] = np.float32

## Initialization

There are a few ways to initialize the class. First, calling it with no arguments initializes all variables with default value:

In [ ]:
prm = x3d.Parameters()

You can access a list with all the available variables at the [Api reference](https://xcompact3d-toolbox.readthedocs.io/en/latest/Docstrings.html).

Let's see how it looks like:

In [ ]:
print(prm)

It is possible to access and/or set values afterwards:

In [ ]:
# Reynolds Number
print(prm.re)

# attribute new value
prm.re = 1e6
print(prm.re)

Second, we can specify some values, and let the missing ones be initialized with default value:

In [ ]:
prm = x3d.Parameters(
    filename="example.i3d",
    itype=10,
    nx=129,
    ny=65,
    nz=32,
    xlx=15.0,
    yly=10.0,
    zlz=3.0,
    nclx1=2,
    nclxn=2,
    ncly1=1,
    nclyn=1,
    nclz1=0,
    nclzn=0,
    iin=1,
    istret=2,
    re=300.0,
    init_noise=0.0125,
    inflow_noise=0.0125,
    dt=0.0025,
    ifirst=1,
    ilast=45000,
    irestart=0,
    icheckpoint=45000,
    ioutput=200,
    iprocessing=50,
)

It is easy to write `example.i3d` to disc, just type:

In [ ]:
prm.write()

And finally, it is possible to read the parameters from the disc:

In [ ]:
prm = x3d.Parameters(filename="example.i3d")
prm.load()

The same result is obtained in a more concise way:

In [ ]:
prm = x3d.Parameters(loadfile="example.i3d")

The class can also read the previous parameters format ([se more information here](https://github.com/fschuch/xcompact3d_toolbox/issues/7)):

``` python
prm = x3d.Parameters(loadfile="incompact3d.prm")
```

There are extra objects read and write the raw binary files from XCompact3d on-demand.

* Read a binary field from the disc:

    ``` python
    ux = prm.dataset.load_array("ux-0000.bin")
    ```

* Read the entire time series for a given variable:

    ``` python
    ux = prm.dataset.load_time_series("ux")
    ```
    
* Read all variables for a given snapshot:

    ```python
    snapshot = prm.dataset.load_snapshot(10)
    ```

* Write `xdmf` files, so the binary files can be open in any external visualization tool:

    ``` python
    prm.dataset.write_xdmf()
    ```

* Compute the coordinates, including support for mesh refinement in y:

In [ ]:
prm.get_mesh()

More details about I/O and array manipulations with [xarray](http://xarray.pydata.org/en/stable/index.html) will be included in a new tutorial.

## Traitlets

### Type-checking

All parameters are type-checked, to make sure that they are what `Xcompact3d` expects. Use the cellcode below to see how a `TraitError` pops out when we try:

```python
prm.itype = 10.5
prm.itype = -5
prm.itype = 20
prm.itype = 'sandbox'

```


### Validation

Some parameters, like mesh points (`nx`, `ny` and `nz`), trigger a validation operation when a new value is attributed to them.
Due to restrictions at the FFT library, they must be equal to:

$$
n_i = \left\{ \begin{array}{ll} 2^{1+a} \times 3^b \times 5^c &\mbox{if periodic,} \\ 2^{1+a} \times 3^b \times 5^c + 1 &\mbox{otherwise,}
\end{array} \right.
$$

where $a$, $b$ and $c$ are non negative integers. In addition, the derivatives stencil imposes that:

$$
n_i \ge \left\{ \begin{array}{ll} 8 &\mbox{if periodic,} \\ 9 &\mbox{otherwise.}
\end{array} \right.
$$

Again, give it a try at the cellcode below:

```python
prm.nx = 129
prm.nx = 4
prm.nx = 60
prm.nx = 61
```

### Observation

Other parameters, like mesh resolution (`dx`, `dy` and `dz`), are automaticaly updated when any new atribution occurs to mesh points and/or domain size. Let's create a quick print functions to play with:

In [ ]:
def show_param():
    for var in "nclx1 nclxn nx xlx dx".split():
        print(f"{var:>5} = {getattr(prm, var)}")

We are starting with:

In [ ]:
show_param()

Let's change just the domain's length:

In [ ]:
prm.xlx = 50.0

show_param()

The resolution was updated as well. Now the number of mesh points:

In [ ]:
prm.nx = 121

show_param()

Again, the resolution was updated. Now we set a new mesh resolution, this time, `xlx` will be updated in order to satisfy the new resolution:

In [ ]:
prm.dx = 1e-2

show_param()

Boundary conditions are observed as well. Xcompact3d allows three different BC for velocity:

* Periodic `0`;
* Free-slip `1`;
* Dirichlet `2`.

They can be assigned individualy for each of the six boundaries:

* `nclx1` and `nclxn`, where $x=0$ and $x=xlx$;
* `ncly1` and `nclyn`, where $y=0$ and $y=yly$;
* `nclz1` and `nclzn`, where $z=0$ and $z=zlz$.

It leads to 5 possibilities (`00`, `11`, `12`, `21` and `22`), because both boundary must be periodic, or not, so `0` cannot be combined.

Let's check it out, we are starting with:

In [ ]:
show_param()

We will change just one side to periodic (`nclx1 = 0`), for consistence, the other side should be periodic too. Let's see:

In [ ]:
prm.nclx1 = 0

show_param()

Now free-slip in one side (`nclx1 = 1`), and the other should be non-periodic:

In [ ]:
prm.nclx1 = 1

show_param()

Setting the other boundary to periodic:

In [ ]:
prm.nclxn = 0

show_param()

and now back to Dirichlet:

In [ ]:
prm.nclxn = 2

show_param()

This time, free-slip:

In [ ]:
prm.nclxn = 1

show_param()

There was no need to update `nclx1`, because `1` and `2` can be freely combined. Notice that `nx` was modified properly from 121 to 120 and then back, according to the possible values, `dx` and `xlx` stayed untouched.

### Metadata

Traitlets types constructors have a `tag` method to store metadata in a dictionary. In the case of Xcompact3d-toolbox, two are especially useful:

* `group` defines to what namespace a given parameter belongs when the class is written to `.i3d` file (`.write()` method) or read from `.i3d` or `.prm` files (`.load()` method), parameters without a group are ignored for both methods;
* `desc` contains a brief description of each parameter that is shown on screen as we saw above, and also printed with the `.write()` method.

### Declaring new parameters

You probably would like to add more parameters for your own flow configuration, or because some of them were not implemented yet (it is a work in progress).

To do so, any auxiliar varible can be included after initialization, like:

In [ ]:
prm.my_variable = 0  # or any other datatype

It was called auxiliar variable because, in this way, it will be available only for the Python aplication.

In order to include it at the `.i3d` file and make it available for XCompact3d, we can create a subclass that inherits all the functionality from `xcompact3d_toolbox.Parameters`:

In [ ]:
import traitlets


# Create a class named my_Parameters, which inherits the properties all properties and methods
class my_Parameters(x3d.Parameters):
    # .tag with group and description guarantees that the new variable will
    # be compatible with all functionalities (like .write() and .load())
    a_my_variable = traitlets.Int(default_value=0, min=0).tag(
        group="BasicParam", desc="An example at the Tutorial <------"
    )

    # And a custom method, for instance
    def my_method(self):
        return self.a_my_variable * 2


prm = my_Parameters(nx=257, ny=129, nz=31, a_my_variable=10)  # and here we go

# Testing the method
print(prm.my_method())

# Show all parameters on screen
print(prm)

Take a look at the source code of [parameters.py](https://github.com/fschuch/xcompact3d_toolbox/blob/master/xcompact3d_toolbox/param.py) if you need more examples for different datatypes.

## Graphical User Interface

<div class="alert alert-info">

For an interactive experience [launch this tutorial on Binder](https://mybinder.org/v2/gh/fschuch/xcompact3d_toolbox/main?labpath=.%2Fdocs%2Fexamples), the widgets are not so responsive when disconnected from a Python application.

</div>

To conclude this part of the tutorial, let's see another option to handle the parameters. The class `ParametersGui` is a subclass of `Parameters`, and includes all the features described above. In addition, `ParametersGui` offers an user interface with [IPywidgets](https://ipywidgets.readthedocs.io/en/stable/index.html).

It is still under development, more parameters and features are going to be included soon, as well as more widgets.

Just like before, we start with:

In [ ]:
prm = x3d.ParametersGui()

Widgets are returned on demand when any instance of `class ParametersGui` is called, let’s see:

In [ ]:
prm("nx", "xlx", "dx", "nclx1", "nclxn")

You can play around with the widgets above and see the effect of the observations made previously.

Notice that the [Traitlets](https://traitlets.readthedocs.io/en/stable/index.html) parameters are related to the value at their widgets in a **two-way link**, in this way, a print will show the actual value on the widgets:

In [ ]:
show_param()

Give it a try, modify the values at the widgets and print them again.

It also works on the other way, set a new value to a parameters will change its widget, try it:

In [ ]:
#prm.nclx1 = 0

And of course, different widgets for the same parameter are always synchronized, change the widget below and see what happens with the widget above:

In [ ]:
prm('nx')

A last example is about the domain decomposition for parallel computation, Xcompact3d uses [2DECOMP&FFT](http://www.2decomp.org/).
The available options for `p_row` and `p_col` are presented as functions of the number of computational cores `ncores`, notice that `p_row * p_col = ncores` should be respected and `p_row * p_col = 0` activates the auto-tunning mode. The widgets are prepared to respect these restrictions:

In [ ]:
prm('ncores', 'p_row', 'p_col')

To conclude this part of the tutorial, let’s see what happens when `class ParametersGui` is presented on screen, hover the mouse over some variable to see its description:

In [ ]:
prm